In [ ]:
from dkube.sdk import *
import os

In [ ]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
user = os.getenv('USERNAME')
code_name = "steel"
steel_data = "steel-data"
prep_ds  = "steel-prep"
train_ds = "steel-train"
test_ds  = "steel-test"
model_name = "steel-model"

In [ ]:
api = DkubeApi(token=existing_token)

In [ ]:
import os, json
import kfp
import kfp.compiler as compiler
import random
import string

generate = lambda hint: "{}-{}".format(hint, ''.join([random.choice(string.digits) for n in range(4)]))
runid = 1

In [ ]:
client = kfp.Client(existing_token=existing_token)

In [ ]:
image = "docker.io/ocdr/d3-datascience-tf-cpu:v1.14-1"
inp_path = "/opt/dkube/input"
out_path = "/opt/dkube/output"
train_ds_path = "/opt/dkube/output/train"
test_ds_path = "/opt/dkube/output/test"
model_inp_path = "/opt/dkube/model"
prep_script = "python preprocessing.py"
split_script = "python split.py"
train_script = "python training.py"
transformer_script = "tf/segmentation/steel-defect/transformer.py"
serving_image = "ocdr/tensorflowserver:2.0.0"
framework = "tensorflow"
f_version = "1.14"

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_preprocess_op = kfp.components.load_component_from_file(components_url + "preprocess/component.yaml")
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-steel-pl',
    description='example steel pipeline'
)
def steel_pipeline(token):
    
    prep = dkube_preprocess_op(token, json.dumps({"image": image}),
                               program=code_name, run_script=prep_script,
                               datasets=json.dumps([steel_data]), input_dataset_mounts=json.dumps([inp_path]),
                               outputs=json.dumps([prep_ds]), output_mounts=json.dumps([out_path])
                              ).set_display_name("Preprocessing")
    
    
    split = dkube_preprocess_op(token, json.dumps({"image": image}),
                               program=code_name, run_script=split_script,
                               datasets=json.dumps([prep_ds]), input_dataset_mounts=json.dumps([inp_path]),
                               outputs=json.dumps([train_ds, test_ds]), output_mounts=json.dumps([train_ds_path, test_ds_path]),
                               ).after(prep).set_display_name("Split")

    train = dkube_training_op(token, json.dumps({"image": image}),
                              framework=framework, version=f_version,
                              program=code_name, run_script=train_script,
                              datasets=json.dumps([str(train_ds)]), outputs=json.dumps([str(model_name)]),
                              input_dataset_mounts=json.dumps([inp_path]),
                              output_mounts=json.dumps([out_path])).after(split).set_display_name("Training")
    
    serving = dkube_serving_op(token, train.outputs['artifact'],
                              device="cpu", serving_image=json.dumps({"image": serving_image}),
                              transformer_image=json.dumps({"image": image}),
                              transformer_project=code_name,
                              transformer_code=transformer_script).after(train).set_display_name("Serving")

In [ ]:
experiment_name = 'Dkube-Steel-pl'
train_experiment = client.create_experiment(name=experiment_name)

In [ ]:
import kfp.compiler as compiler
compiler.Compiler().compile(steel_pipeline, "steel-pipeline.zip")
try:
    pipeline = client.upload_pipeline("steel-pipeline.zip", pipeline_name = "Steel-pipeline")
except BaseException as e:
    print(e)
runid = 1

In [ ]:
arguments = {"token":existing_token}
run = client.run_pipeline(train_experiment.id, job_name=f"[{pipeline.name}] Run" + str(runid), pipeline_id=pipeline.id, params=arguments)
runid += 1